## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-21-18-59-55 +0000,nyt,64% of Americans Disapprove of Trump’s Tariffs,https://www.nytimes.com/2026/02/20/us/politics...
1,2026-02-21-18-57-07 +0000,nyt,"At Least 10 Killed in Lebanon, Officials Say, ...",https://www.nytimes.com/2026/02/21/world/middl...
2,2026-02-21-18-56-29 +0000,nyt,Klaebo Makes Olympic History: Unforgettable Ph...,https://www.nytimes.com/2026/02/21/world/johan...
3,2026-02-21-18-46-03 +0000,startribune,United States vs. Canada will have a winner: e...,https://www.startribune.com/united-states-vs-c...
4,2026-02-21-18-39-18 +0000,nypost,News of the World: What you missed this week i...,https://nypost.com/2026/02/21/world-news/news-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2457/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
2,trump,66
40,court,42
172,supreme,36
3,tariffs,30
66,tariff,25


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
184,2026-02-21-00-24-00 +0000,wsj,President Trump had just started a closed-door...,https://www.wsj.com/politics/policy/the-moment...,292
117,2026-02-21-10-30-00 +0000,wsj,The Supreme Court’s ruling that most of Trump’...,https://www.wsj.com/economy/trade/trump-lost-o...,229
48,2026-02-21-16-56-26 +0000,cbc,Trump says he's raising new global tariff rate...,https://www.cbc.ca/news/politics/trump-global-...,221
192,2026-02-20-23-26-00 +0000,wsj,The Supreme Court decision to strike down Pres...,https://www.wsj.com/economy/trade/trump-tariff...,221
237,2026-02-20-20-56-39 +0000,wapo,"Trump imposes new tariffs, denounces Supreme C...",https://www.washingtonpost.com/national-securi...,220


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
184,292,2026-02-21-00-24-00 +0000,wsj,President Trump had just started a closed-door...,https://www.wsj.com/politics/policy/the-moment...
48,72,2026-02-21-16-56-26 +0000,cbc,Trump says he's raising new global tariff rate...,https://www.cbc.ca/news/politics/trump-global-...
180,57,2026-02-21-00-49-41 +0000,nypost,Los Angeles small business owner takes wait-an...,https://nypost.com/2026/02/20/business/los-ang...
257,33,2026-02-20-19-56-29 +0000,nyt,Former Prince Andrew’s Arrest Upends Royal Fam...,https://www.nytimes.com/2026/02/20/world/europ...
32,29,2026-02-21-17-36-48 +0000,nyt,"After Tariff Decision, What Happens to Trump’s...",https://www.nytimes.com/2026/02/20/business/ec...
74,29,2026-02-21-14-21-18 +0000,nypost,DOJ axed NY judge who handed out more asylum c...,https://nypost.com/2026/02/21/us-news/youre-fi...
152,29,2026-02-21-03-34-54 +0000,cbc,Here's where U.S. military forces are as they ...,https://www.cbc.ca/player/play/9.7100689?cmp=rss
119,27,2026-02-21-10-05-15 +0000,nypost,Justice Department swiftly fires lawyer chosen...,https://nypost.com/2026/02/21/us-news/virginia...
71,27,2026-02-21-14-50-46 +0000,nypost,Trans sex predator—who switched genders on tax...,https://nypost.com/2026/02/21/world-news/sex-p...
75,26,2026-02-21-14-20-38 +0000,nypost,"Aloe, nice to meet you! New AI device lets yo...",https://nypost.com/2026/02/21/us-news/amazing-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
